# 🧠 Transfer Learning with PyTorch

Welcome to this beginner-friendly notebook where we'll explore **Transfer Learning** using **PyTorch**. 

### 👇 What you'll learn:
- What is transfer learning?
- How to use a pretrained model (ResNet18)
- How to replace the classifier head
- How to train it on a small dataset (e.g., CIFAR-10 subset)

## 🔍 Step 1: Import Libraries

## 📥 Step 2: Load Dataset (CIFAR-10 Subset)

## 🧠 Step 3: Load Pretrained ResNet18 and Modify Classifier

##  Function to Calculate the parameters

In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params



## Info and Parameters

## 🧠 Step 3:  Modify Classifier

## 🏋️‍♂️ Step 4: Train the New Classifier

## ✅ Conclusion
- We used a **pretrained ResNet18** as a **feature extractor**.
- We replaced the top layer and trained it on a small dataset.
- You just performed **transfer learning** in PyTorch! 🎉